In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pde
import os


[[[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  ...

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   ...
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   ...
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.  0.]]

  [[ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   ...
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.  0.]]

  [[ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   ...
   [ 1.  0.  0.]
   [ 1.  0.  0.]
   [ 1.  0.

# Boundary conditions test

In [142]:
class CustomPDE(pde.PDEBase):

    def __init__(self, bc):
        array_shape = (20, 10, 10)
        # self.x_component_array = np.arange(array_shape[0]).reshape(array_shape[0], 1, 1) * np.ones(array_shape[1:])
        self.bc = bc
        # self.boundary_mask = boundary_mask

        # array_shape = (20, 10, 10)
        # x_component_array = np.arange(array_shape[0]).reshape(array_shape[0], 1, 1) * np.ones(array_shape[1:])

        # # Create a new array for the vectors
        # self.vector_array = np.zeros((3, 20, 10, 10))

        # # Fill in the x-component and set y and z components to zero
        # self.vector_array[0] = x_component_array  # x component
        # y and z components remain zero by default
        self.vector_array = np.ones((3, 20, 10, 10))

    def evolution_rate(self, state, t=0):
        """ Custom PDE evolution with modified diffusion rate """
        # laplacian = state.laplace(bc={"value": 0})  # zero Dirichlet boundary condition
        # rate = laplacian.copy()  # Default rate inside the grid
        
        # Reduce the rate of diffusion at grid to a quarter
        # rate.data[self.boundary_mask] *= 0.25
        shear_viscosity = 2E-5
        bulk_viscosity = 5E-5
        f_u = state.dot(state.gradient(bc=self.bc)) - shear_viscosity * state.laplace(bc=self.bc) \
                - (bulk_viscosity + shear_viscosity/3) * state.divergence(bc=self.bc).gradient(bc=self.bc) 
        
        return -f_u - 0.00001 * self.vector_array


In [143]:
grid = pde.CartesianGrid([[0, 10], [0, 5], [0, 5]], [20, 10, 10], periodic=[False, False, False])
field = pde.VectorField(grid, data=0)

bc_x = ({"neumann": 0}, {"value": 0})

eq = CustomPDE(bc=[bc_x, bc_x, bc_x])
result = eq.solve(field, t_range=20, dt=0.1)
result.to_scalar(scalar='norm').plot_interactive()













  0%|          | 0/20.0 [1:15:36<?, ?it/s]
  0%|          | 0/20.0 [12:28<?, ?it/s]
  0%|          | 0/20.0 [10:30<?, ?it/s]
  0%|          | 0/20.0 [09:56<?, ?it/s]




















100%|██████████| 20.0/20.0 [00:01<00:00, 14.22it/s]


In [78]:
print(result.data[-1])

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[-5.37511274e-09  1.42198632e-20  0.00000000e+00 ...  0.00000000e+00
   -1.42198632e-20  5.37511274e-09]
  [-5.37511258e-09  1.42176097e-20  0.00000000e+00 ...  0.00000000e+00
   -1.42176097e-20  5.37511258e-09]
  [-5.37511258e-09  1.42176097e-20  0.00000000e+00 ...  0.00000000e+00
   -1.42176097e-20  5.37511258e-09]
  ...
  [-5.37511258e-09  1.4

In [63]:
grid = pde.CartesianGrid([[0, 10], [0, 5], [0, 5]], [20, 10, 10], periodic=[False, False, False])
field = pde.VectorField(grid, data=0)
field.insert((5,5,5), [0.001,0,0])
field.to_scalar(scalar='norm').plot_interactive()
eq = CustomPDE(boundary_mask=False)
result = eq.solve(field, t_range=60, dt=0.1)
result.to_scalar(scalar='norm').plot_interactive()

100%|██████████| 60.0/60.0 [00:00<00:00, 241.34it/s]  


# JIT test

In [247]:
from pde.tools.numba import jit

In [248]:

class KuramotoSivashinskyPDE(pde.PDEBase):

    def __init__(self, diffusivity=1, bc="auto_periodic_neumann", bc_laplace="auto_periodic_neumann"):
        """ initialize the class with a diffusivity and boundary conditions
        for the actual field and its second derivative """
        self.diffusivity = diffusivity
        self.bc = bc
        self.bc_laplace = bc_laplace


    def evolution_rate(self, state, t=0):
        """ numpy implementation of the evolution equation """
        state_lapacian = state.laplace(bc=self.bc)
        state_gradient = state.gradient(bc="auto_periodic_neumann")
        return (- state_lapacian.laplace(bc=self.bc_laplace)
                - state_lapacian
                - 0.5 * self.diffusivity * (state_gradient @ state_gradient))


    def _make_pde_rhs_numba(self, state):
        """ the numba-accelerated evolution equation """
        # make attributes locally available
        diffusivity = self.diffusivity

        # create operators
        laplace_u = state.grid.make_operator("laplace", bc=self.bc)
        gradient_u = state.grid.make_operator("gradient", bc=self.bc)
        laplace2_u = state.grid.make_operator("laplace", bc=self.bc_laplace)
        dot = pde.VectorField(state.grid).make_dot_operator()
        dim = state.grid.dim

        @jit
        def pde_rhs(state_data, t=0):
            """ compiled helper function evaluating right hand side """
            state_lapacian = laplace_u(state_data)
            state_grad = gradient_u(state_data)
            result = - laplace2_u(state_lapacian) - state_lapacian

            for i in range(state_data.size):
                for j in range(dim):
                    result.flat[i] -= diffusivity / 2 * state_grad[j].flat[i]**2

            return result

        return pde_rhs

In [249]:
grid_size = [64, 64]
grid = pde.UnitGrid(grid_size, periodic=True)
state = pde.ScalarField(grid, data=np.random.rand(*grid.shape))

eq = KuramotoSivashinskyPDE(diffusivity=0.1)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=33, dt=0.001, tracker=storage.tracker(0.1))

In [250]:
# Plot calculated results
# WARNING: This will take a long time to run, uncomment with care

# pde.movie(storage, filename="output2.mp4", plot_args={"vmin": -30, "vmax": 30})

# 3D test

In [251]:
# generate grid
grid_size = [64, 64, 64]
grid = pde.UnitGrid(grid_size)  
state = pde.ScalarField(grid, data=0)
state.insert([32, 32, 32], 1)  # insert peak 

# Define the mask for grid lines with thickness of 5 in 3D
x, y, z = grid.cell_coords[..., 0], grid.cell_coords[..., 1], grid.cell_coords[..., 2]
section_size = grid_size[0] // 3
boundary_mask = (
    ((x % section_size <= 2) | (x % section_size >= section_size - 2)) |
    ((y % section_size <= 2) | (y % section_size >= section_size - 2)) |
    ((z % section_size <= 2) | (z % section_size >= section_size - 2))
)

eq = CustomPDE(boundary_mask)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=30, dt=0.1, tracker=storage.tracker(0.1))


In [267]:
# Plot calculated results in 3D
# WARNING: This will take even longer time to run, uncomment with care

# slice at depth 32
# print(storage[0].data[32].shape)
# print(storage[0].data[32])
# print(storage[0].data[:,:,32])
# print(storage[0])

# @jit
def get_slice(storage, z=32):
    # Transform the 3D storage data to 2D slice at depth z
    grid_size = [64, 64]
    grid = pde.UnitGrid(grid_size)  
    
    new_data = []
    for time in range(len(storage)):
        data=storage[time].data[:,:,z]
        new_data.append(data)
    new_data = np.array(new_data)
    field_obj = pde.ScalarField(grid, data=new_data[0])
    res = pde.storage.memory.MemoryStorage(times=list(range(len(storage))), data=new_data, field_obj=field_obj)
    print("res",res[0])
        
    return res


new_storage = get_slice(storage, 28)
pde.movie(new_storage, filename="output3.mp4", plot_args={"vmin": 0, "vmax": 0.01,})

res ScalarField(grid=UnitGrid(shape=(64, 64), periodic=[False, False]), data=Array(64, 64))


100%|██████████| 300/300 [00:06<00:00, 44.20it/s]
